In [2]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 960.0 kB/s eta 0:02:11
   ---------------------------------------- 0.2/124.9 MB 2.6 MB/s eta 0:00:48
   ---------------------------------------- 0.4/124.9 MB 3.5 MB/s eta 0:00:36
   ---------------------------------------- 0.6/124.9 MB 3.8 MB/s eta 0:00:33
   ---------------------------------------- 0.8/124.9 MB 4.0 MB/s eta 0:00:31
   ---------------------------------------- 1.0/124.9 MB 4.1 MB/s eta 0:00:30
   ---------------------------------------- 1.3/124.9 MB 4.2 MB/s eta 0:00:30
    --------------------------------------- 1.6/124.9 MB 4.5 MB/s eta 0:00:28
    --------------------------------------- 1.9/124.9 MB 4.8 MB/s eta 0:00:26
    --------------------------------------- 2.2/124.9 MB 5.1 MB/s eta 0:00:24
    --------------------------------------- 2.5/124.9 MB 5.3 MB/s eta 0:00:23
    --------------------------------------- 2.7/124.9 MB 5.4 MB/s eta

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
阶段2: 嵌套交叉验证多模型比较（Logistic / RF / SVM / XGBoost）
- 外层CV: 5-Fold StratifiedKFold (重复3次)
- 内层CV: 5-Fold StratifiedKFold (超参数调优)
- Pipeline: StandardScaler + SelectKBest + Classifier
- 模型: ElasticNet LogisticRegression, RandomForest, SVM, XGBoost
"""

import pandas as pd
import numpy as np
import os
import json
import pickle
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, 
                             f1_score, recall_score, precision_score,
                             accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# 配置路径
BASE_DIR = "D:/PHM5005/5005-main"
RESULTS_DIR = os.path.join(BASE_DIR, "results")
OUTPUT_DIR = os.path.join(RESULTS_DIR, "stage2_nested_cv_multimodel")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=" * 80)
print("阶段2: 嵌套交叉验证多模型训练")
print("=" * 80)

# ============================================================================
# 1. 加载数据
# ============================================================================
X_train = np.load(os.path.join(RESULTS_DIR, 'X_train.npy'), allow_pickle=True)
X_test = np.load(os.path.join(RESULTS_DIR, 'X_test.npy'), allow_pickle=True)
y_train = np.load(os.path.join(RESULTS_DIR, 'y_train.npy'), allow_pickle=True)
y_test = np.load(os.path.join(RESULTS_DIR, 'y_test.npy'), allow_pickle=True)

with open(os.path.join(RESULTS_DIR, 'feature_names.json'), 'r') as f:
    feature_names = json.load(f)['all_features']

print(f"  训练集: {X_train.shape}, 测试集: {X_test.shape}")

# ============================================================================
# 2. 定义评估函数
# ============================================================================
def calculate_metrics(y_true, y_pred, y_proba):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return {
        'auroc': roc_auc_score(y_true, y_proba),
        'auprc': average_precision_score(y_true, y_proba),
        'f1': f1_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'accuracy': accuracy_score(y_true, y_pred)
    }

# ============================================================================
# 3. 定义多个模型的 Pipeline 和超参数网格
# ============================================================================
print("\n[2] 定义Pipeline和超参数网格...")

### ← 修改点：增加多个模型
models = {
    "Logistic": {
        "pipeline": Pipeline([
            ('scaler', StandardScaler()),
            ('selector', SelectKBest(f_classif)),
            ('clf', LogisticRegression(
                solver='saga', penalty='elasticnet',
                class_weight='balanced', max_iter=5000, random_state=RANDOM_STATE))
        ]),
        "param_grid": {
            'selector__k': [300, 500, 700, 'all'],
            'clf__l1_ratio': [0.2, 0.5, 0.8],
            'clf__C': [0.01, 0.1, 1, 10]
        }
    },
    "RandomForest": {
        "pipeline": Pipeline([
            ('selector', SelectKBest(f_classif)),
            ('clf', RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE))
        ]),
        "param_grid": {
            'selector__k': [300, 500, 700, 'all'],
            'clf__n_estimators': [200, 500],
            'clf__max_depth': [5, 10, 15, None],
            'clf__min_samples_split': [2, 5, 10],
            'clf__min_samples_leaf': [1, 2, 4]
        }
    },
    "SVM": {
        "pipeline": Pipeline([
            ('scaler', StandardScaler()),
            ('selector', SelectKBest(f_classif)),
            ('clf', SVC(probability=True, class_weight='balanced', random_state=RANDOM_STATE))
        ]),
        "param_grid": {
            'selector__k': [300, 500, 700, 'all'],
            'clf__C': [0.01, 0.1, 1, 10, 100],
            'clf__kernel': ['linear', 'rbf']
        }
    },
    "XGBoost": {
        "pipeline": Pipeline([
            ('selector', SelectKBest(f_classif)),
            ('clf', XGBClassifier(
                objective='binary:logistic', eval_metric='auc',
                use_label_encoder=False, random_state=RANDOM_STATE, n_jobs=-1,
                scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)
            ))
        ]),
        "param_grid": {
            'clf__n_estimators': [300, 500, 700, 'all'],           
            'clf__max_depth': [3, 5, 7, 9],                 
            'clf__learning_rate': [0.01, 0.05, 0.1],       
            'clf__subsample': [0.8, 1.0],                  
            'clf__colsample_bytree': [0.8, 1.0]
        }
    }
}

# ============================================================================
# 4. 嵌套交叉验证
# ============================================================================
outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=RANDOM_STATE)
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

### ← 修改点：多模型循环
for model_name, model_info in models.items():
    print(f"\n{'='*80}\n▶ 模型: {model_name}\n{'='*80}")
    
    model_dir = os.path.join(OUTPUT_DIR, model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    outer_results, best_params_history = [], []
    
    for fold_idx, (train_idx, val_idx) in enumerate(tqdm(outer_cv.split(X_train, y_train), total=15, desc=f"{model_name} 外层CV"), 1):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        grid = GridSearchCV(
            model_info["pipeline"],
            model_info["param_grid"],
            cv=inner_cv, scoring='roc_auc', n_jobs=-1, verbose=0
        )
        grid.fit(X_tr, y_tr)
        
        best_model = grid.best_estimator_
        best_params_history.append(grid.best_params_)
        
        y_pred = best_model.predict(X_val)
        y_proba = best_model.predict_proba(X_val)[:, 1]
        
        metrics = calculate_metrics(y_val, y_pred, y_proba)
        metrics['fold'] = fold_idx
        outer_results.append(metrics)
    
    # 外层CV结果汇总
    df = pd.DataFrame(outer_results)
    summary = df.mean().to_dict()
    
    print(f"AUROC: {summary['auroc']:.3f} ± {df['auroc'].std():.3f}, AUPRC: {summary['auprc']:.3f}")
    
    # 保存结果
    df.to_csv(os.path.join(model_dir, "outer_cv_results.csv"), index=False)
    pd.DataFrame(best_params_history).to_csv(os.path.join(model_dir, "best_params.csv"), index=False)
    
    # 在全训练集上拟合并测试
    best_param_df = pd.DataFrame(best_params_history)
    mode_params = best_param_df.mode().iloc[0].to_dict()

    int_params = ['selector__k', 'clf__n_estimators', 'clf__max_depth']
    for p in int_params:
        if p in mode_params:
            if mode_params[p] != 'all' and mode_params[p] is not None:
                mode_params[p] = int(mode_params[p])

    final_pipe = model_info["pipeline"].set_params(**mode_params)
    final_pipe.fit(X_train, y_train)
    y_test_pred = final_pipe.predict(X_test)
    y_test_proba = final_pipe.predict_proba(X_test)[:, 1]
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    
    with open(os.path.join(model_dir, "test_metrics.json"), "w") as f:
        json.dump(test_metrics, f, indent=2)
    
    print(f"测试集 AUROC: {test_metrics['auroc']:.3f}, AUPRC: {test_metrics['auprc']:.3f}")

print("\n✅ 所有模型完成！结果保存在:", OUTPUT_DIR)

C:\Users\Hermit\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


阶段2: 嵌套交叉验证多模型训练
  训练集: (289, 906), 测试集: (73, 906)

[2] 定义Pipeline和超参数网格...

▶ 模型: Logistic


Logistic 外层CV: 100%|█████████████████████████████████████████████████████████████████| 15/15 [05:10<00:00, 20.70s/it]


AUROC: 0.657 ± 0.093, AUPRC: 0.425
测试集 AUROC: 0.702, AUPRC: 0.373

▶ 模型: RandomForest


RandomForest 外层CV: 100%|█████████████████████████████████████████████████████████████| 15/15 [13:16<00:00, 53.09s/it]


AUROC: 0.655 ± 0.090, AUPRC: 0.420
测试集 AUROC: 0.686, AUPRC: 0.362

▶ 模型: SVM


SVM 外层CV: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:24<00:00,  1.60s/it]


AUROC: 0.539 ± 0.154, AUPRC: 0.337
测试集 AUROC: 0.312, AUPRC: 0.196

▶ 模型: XGBoost


XGBoost 外层CV: 100%|██████████████████████████████████████████████████████████████████| 15/15 [02:09<00:00,  8.64s/it]


AUROC: 0.648 ± 0.054, AUPRC: 0.399
测试集 AUROC: 0.567, AUPRC: 0.413

✅ 所有模型完成！结果保存在: D:/PHM5005/5005-main\results\stage2_nested_cv_multimodel
